In [1]:
# need to set up database first from most recent database dump - this should be scripted

In [2]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import os.path

In [3]:
conn = pg.connect("dbname='carlplaying' user='drcjar' password='drcjar_is_not_fake'")

In [4]:
df = psql.read_sql("SELECT * FROM ipfjes_demographics", conn)

In [5]:
df.columns

Index(['id', 'created', 'updated', 'consistency_token', 'hospital_number',
       'nhs_number', 'surname', 'first_name', 'middle_name', 'date_of_birth',
       'religion', 'date_of_death', 'post_code', 'gp_practice_code',
       'death_indicator', 'title_ft', 'marital_status_ft', 'sex_ft',
       'birth_place_ft', 'ethnicity_ft', 'birth_place_fk_id', 'created_by_id',
       'ethnicity_fk_id', 'marital_status_fk_id', 'patient_id', 'sex_fk_id',
       'title_fk_id', 'updated_by_id', 'contact_details', 'phone_number'],
      dtype='object')

In [6]:
df = df[['contact_details', 'hospital_number', 'first_name', 'surname', 'date_of_birth']]

In [7]:
df.contact_details = df.contact_details.str.replace('GP','')
df.contact_details = df.contact_details.str.replace(':','')

In [8]:
save_path = '/home/drcjar/Sync/ipfjes-pid/ipfjes-letters/letters'

def make_letter(hospital_number, gpaddress, first_name, surname, date_of_birth):
    with open('ipfjes-coverletter-gp.tex', 'r') as f:   # this is our template it also calls a .lco file
        letter = f.read()
        letter = letter.replace('gpaddress', gpaddress)
        letter = letter.replace('first_name', first_name)
        letter = letter.replace('surname', surname)
        letter = letter.replace('date_of_birth', date_of_birth)
    with open(os.path.join(save_path, hospital_number + ".tex"), 'w') as f:
        f.write(letter)

In [9]:
for i in range(len(df)):
    try:
        hospital_number = df.iloc[i]['hospital_number']
        gpaddress = df.iloc[i]['contact_details']
        first_name = df.iloc[i]['first_name']
        surname = df.iloc[i]['surname']
        date_of_birth = str(df.iloc[i]['date_of_birth'].strftime('%d/%m/%Y'))
        make_letter(hospital_number, gpaddress, first_name, surname, date_of_birth)
    except (TypeError, AttributeError):
        continue

In [10]:
# to make all the pdf need to run: for file in *.tex; do xelatex -interaction nonstopmode "$file"; done 